In [436]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [437]:
from cloudant.client import Cloudant
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey
from datetime import datetime
from faker import Faker
import random
import json

In [438]:
username = "8c6b43fe-21d2-4fe1-9a35-f16ed4b5bc8c-bluemix"
password = "_BZzLTTJK8Bw3D17DM76Em34GLlzW-_3o5RpA9c1oZnR"
client = Cloudant.iam(username, password)
client.connect()
databaseName = "maindatabase"
mainDatabase = client.create_database(databaseName)

In [439]:
fake = Faker()
Faker.seed(4321)
location_list = []
for i in range(5):
    location_list.append((float(fake.local_latlng()[0]), float(fake.local_latlng()[1])))
print(location_list)

[(35.74788, -88.3162), (30.17746, -76.51552), (39.45621, -116.56346), (29.65163, -76.72803), (28.15112, -96.39774)]


In [440]:
googlePlaceNycHallID = "ChIJs--MqP1YwokRBwAhjXWIHn8"
googlePlaceMuskogeeID = "ChIJRZv8MU7gtYcRKKC5VXD8wXc"
googlePlacePalmValleyID = "ChIJr46dPFY25IgRVZvncDr516U"
googlePlaceCloverHillID = "ChIJJzds9ivbyYkRThGcZ8l4NJQ"
googlePlaceGainesVilleID = "ChIJKyy_aSEg5ogRlo0QKF1554A"
geoJsonNycHall = {"GooglePlaceID": googlePlaceNycHallID, "Coordinates": location_list[0]}
geoJsonMuskogee = {"GooglePlaceID": googlePlaceMuskogeeID, "Coordinates":location_list[1]}
geoJsonPalmValley = {"GooglePlaceID": googlePlacePalmValleyID, "Coordinates":location_list[2]}
geoJsonCloverHill = {"GooglePlaceID": googlePlaceCloverHillID, "Coordinates":location_list[3]}
geoJsonGainesVille = {"GooglePlaceID": googlePlaceGainesVilleID,"Coordinates":location_list[4]}

geoJsonList = []
geoJsonList.append(geoJsonNycHall)
geoJsonList.append(geoJsonMuskogee)
geoJsonList.append(geoJsonPalmValley)
geoJsonList.append(geoJsonCloverHill)
geoJsonList.append(geoJsonGainesVille)
print(geoJsonList)



[{'GooglePlaceID': 'ChIJs--MqP1YwokRBwAhjXWIHn8', 'Coordinates': (35.74788, -88.3162)}, {'GooglePlaceID': 'ChIJRZv8MU7gtYcRKKC5VXD8wXc', 'Coordinates': (30.17746, -76.51552)}, {'GooglePlaceID': 'ChIJr46dPFY25IgRVZvncDr516U', 'Coordinates': (39.45621, -116.56346)}, {'GooglePlaceID': 'ChIJJzds9ivbyYkRThGcZ8l4NJQ', 'Coordinates': (29.65163, -76.72803)}, {'GooglePlaceID': 'ChIJKyy_aSEg5ogRlo0QKF1554A', 'Coordinates': (28.15112, -96.39774)}]


In [441]:
def randomizeCondition() :
    num = random.randint(0, 2)
    if (num == 0) :
        return "Recovered/NoCovid19"
    if (num == 1) :
        return "PotentialCovid19"
    if (num == 2) :
        return "HasCovid19"
status = randomizeCondition()

def randomizeVisitID() :
    
    return random.randint(0, 999999999)

visit_id = randomizeVisitID()
print(visit_id)

customListVisitDictionary = [{str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]},
                            {str(geoJsonNycHall["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]}]


def initListVisitDictionary(geoJsonList) :
    visitListDictionary = []
    for i in range(5):
        visitListDictionary.append({str(geoJsonList[i]["Coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]})     
    return visitListDictionary

ListVisitDictionary = initListVisitDictionary(geoJsonList)


def randomizeTransactionID() :
    return random.randint(0, 32800000)

def randomizeContributionID() :
    return random.randint(0, 32800000)

def init_ListJsonDocument(ListVisitDictionary, geoJsonList):
    ListJsonDocument = []
    for i in range(len(geoJsonList)):
        jsonDocument = {
            "TransactionID" : randomizeTransactionID(),
            "ContributorID" : randomizeContributionID(),
            "GeoLocation": geoJsonList[i],
            "VisitDictionary": ListVisitDictionary[i]     
        }
        type(jsonDocument)
        ListJsonDocument.append(jsonDocument)
    return ListJsonDocument



79856523


In [442]:
print(ListVisitDictionary)

[{'(35.74788, -88.3162)': [296258854, '2020-03-13 00:00:00', 'HasCovid19']}, {'(30.17746, -76.51552)': [620237565, '2020-03-13 00:00:00', 'HasCovid19']}, {'(39.45621, -116.56346)': [152839336, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'(29.65163, -76.72803)': [535667885, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'(28.15112, -96.39774)': [894694269, '2020-03-13 00:00:00', 'PotentialCovid19']}]


In [443]:
ListJsonDocument = init_ListJsonDocument(ListVisitDictionary, geoJsonList)
type(customListVisitDictionary)


list

In [444]:
def initListIBMDataBaseDocument(ListJsonDocument) :
    ListIBMDataBaseDocument = []
    for i in ListJsonDocument:
        newDocument = mainDatabase.create_document(i)
        ListIBMDataBaseDocument.append(newDocument)
    return ListIBMDataBaseDocument
ListIBMDataBaseDocument = initListIBMDataBaseDocument(ListJsonDocument)

In [445]:
print(ListIBMDataBaseDocument)

[{'TransactionID': 7226532, 'ContributorID': 20832034, 'GeoLocation': {'GooglePlaceID': 'ChIJs--MqP1YwokRBwAhjXWIHn8', 'Coordinates': (35.74788, -88.3162)}, 'VisitDictionary': {'(35.74788, -88.3162)': [296258854, '2020-03-13 00:00:00', 'HasCovid19']}, '_id': 'ee51920e43b3f048c60c0c3c3207f978', '_rev': '1-b269259268af218f5bff5579343b1f32'}, {'TransactionID': 19914301, 'ContributorID': 8708241, 'GeoLocation': {'GooglePlaceID': 'ChIJRZv8MU7gtYcRKKC5VXD8wXc', 'Coordinates': (30.17746, -76.51552)}, 'VisitDictionary': {'(30.17746, -76.51552)': [620237565, '2020-03-13 00:00:00', 'HasCovid19']}, '_id': '83e7558b1745f0df1a17bce3511110ef', '_rev': '1-bc86f19abde5443372915b6313fe9a42'}, {'TransactionID': 16597127, 'ContributorID': 4728590, 'GeoLocation': {'GooglePlaceID': 'ChIJr46dPFY25IgRVZvncDr516U', 'Coordinates': (39.45621, -116.56346)}, 'VisitDictionary': {'(39.45621, -116.56346)': [152839336, '2020-03-13 00:00:00', 'PotentialCovid19']}, '_id': 'cc29fb9cf6a700d6dccd470bbab29f00', '_rev': '1-

In [446]:
def severityOfArea(ListVisitDictionary, geoJsonObject) :
    locationSeverityDict = {}
    highCount = 0
    mediumCount = 0
    lowCount = 0
    highWeight = 1
    mediumWeight = 0.5
    lowWeight = 0
    weightedSum = 0
    for subDict in ListVisitDictionary:
        print(subDict)
        for key in subDict:
            strGeoJsonCoordinates = str(geoJsonObject["Coordinates"])
            if (key == strGeoJsonCoordinates):
                if (subDict[key][2].lower() == "HasCovid19".lower()):
                    highCount = highCount + 1
                elif (subDict[key][2].lower() == "PotentialCovid19".lower()):
                    mediumCount = mediumCount + 1
                else:
                    lowCount = lowCount + 1
    weightedSum = ((1 * highCount) + (0.5 * mediumCount) + (lowCount * lowWeight ))
    locationSeverityDict[geoJsonObject["Coordinates"]] = weightedSum
    return locationSeverityDict

LocationSeverityDict = severityOfArea(customListVisitDictionary, geoJsonNycHall)
print(LocationSeverityDict)

{'(35.74788, -88.3162)': [829090750, '2020-03-13 00:00:00', 'PotentialCovid19']}
{'(35.74788, -88.3162)': [457098481, '2020-03-13 00:00:00', 'PotentialCovid19']}
{'(35.74788, -88.3162)': [977816415, '2020-03-13 00:00:00', 'PotentialCovid19']}
{'(35.74788, -88.3162)': [555277930, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}
{'(35.74788, -88.3162)': [677509750, '2020-03-13 00:00:00', 'HasCovid19']}
{(35.74788, -88.3162): 2.5}
